文本预处理，将所有训练数据合并成一个文件，去掉标点，停词

In [1]:
import re,os
import numpy as np
import pandas as pd
np.random.seed(42)

In [2]:
INPUT_PATH = '../input/'
CACHE_PATH = '../cache/'
OUTPUT_PATH ='../output'

In [3]:
# 读取数据
df_train_first = pd.read_csv(INPUT_PATH + 'train_first.csv')
df_train_second = pd.read_csv(INPUT_PATH + 'train_second.csv')
df_predict_first = pd.read_csv(INPUT_PATH + 'predict_first.csv')
df_predict_second = pd.read_csv(INPUT_PATH + 'predict_second.csv')

In [4]:
# 合并数据
df_train = pd.concat([df_train_first,df_train_second,df_predict_first],axis=0,ignore_index=True)
df_predict = df_predict_second.copy()

In [5]:
import os
LTP_DATA_DIR = '../ltp_data_v3.4.0/'  # ltp模型目录的路径
cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`

In [6]:
from pyltp import Segmentor
segmentor = Segmentor()  # 初始化实例
segmentor.load_with_lexicon(cws_model_path, './china_position')  # 加载模型，第二个参数是您的外部词典文件路径


In [7]:
def hgda_nlp(text):

    words = segmentor.segment(text)
    # segmentor.release()
    return words


In [8]:
# 预处理单个词
def preprocess_word(word):
    # 去掉标点
    word = word.strip('\'"?!,.():;？！，。…“”（）：；<>《》/ 、rn【】[]|~#%&*br')
    # 去掉 - &
    word = re.sub(r'(-|\')', '', word)
    
    return word

In [9]:
# 预处理句子
def preprocess_sentence(sentence, stop_words=False):
    sentence = sentence.lower()
    # 转化URL
    sentence = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', sentence)
    # 去掉空白，单双引号
    sentence = sentence.strip(' "\'')
    # 分词
    words = hgda_nlp(sentence)
    # 分别处理每一个单词
    sentence = []
    for word in words:
        word = preprocess_word(word)
        if len(word) > 0:
            sentence.append(word)
    if len(sentence) == 0:
        return ' '.join(list('很好'))
    else:
        return ' '.join(sentence)

In [10]:
# 拷贝数据
df_train_processed = df_train.copy()
df_predict_processed = df_predict.copy()

In [11]:
# 预处理文本
df_train_processed['Discuss'] = df_train['Discuss'].map(preprocess_sentence)
df_predict_processed['Discuss'] = df_predict['Discuss'].map(preprocess_sentence)

In [12]:
df_train_processed

,Discuss,Id,Score
0,好 大 的 一个 游乐 公园 已经 去 了 2 次 但 感觉 还 没有 玩 够 似的 会 有...,201e8bf2-77a2-3a98-9fcf-4ce03914e712,5.0
1,新 中国 成立 也 是 在 这 举行 对 我们 中国 人 来说 有些 重要 及 深刻 的 意义,f4d51947-eac4-3005-9d3c-2f32d6068a2d,4.0
2,庐山 瀑布 非常 有名 也 有 非常 多 个 瀑布 只是 最 好看 的 非三叠泉莫属 推荐 一 去,74aa7ae4-03a4-394c-bee0-5702d3a3082a,4.0
3,个人 觉得 颐和园 是 北京 最 值 的 一起 的 地方 不过 相比 下 门票 也 是 最 ...,099661c2-4360-3c49-a2fe-8c783764f7db,5.0
4,迪斯尼 一 日 游,97ca672d-e558-3542-ba7b-ee719bba1bab,5.0
5,方便,3b7f3f2e-886f-3a68-a810-2c37cfd728d3,4.0
6,看 水 看 山 都 可以 感受 古人 的 智慧 结晶 秋景 美丽 如 画 红 黄 绿 相间 ...,88914409-bd13-3d47-b5a2-691177dde8fd,4.0
7,赞,bf13ec92-6079-3451-ade3-88020cb0dcb5,5.0
8,唯一 糟点,489c3d94-9c44-3cf2-949c-1b507c374c69,5.0
9,周 未 周边 游,285bba78-16a3-3c1d-b648-baa483883ee3,5.0


In [13]:
# 保存预处理后的数据集
df_train_processed.to_csv(CACHE_PATH + 'train_processed_all.csv',index=False)
df_predict_processed.to_csv(CACHE_PATH+ 'predict_processed_all.csv',index=False)